In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 550.1 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [ ]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [ ]:
!pip install moviepy pydub


In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import os
import time
from google.colab.patches import cv2_imshow
from pydub import AudioSegment
from pydub.generators import Sine
from moviepy.editor import VideoFileClip, concatenate_audioclips, AudioFileClip, CompositeAudioClip

model=YOLO('yolov8s.pt')


class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
              'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
              'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
              'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
              'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
              'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
              'cell phone', 'microwave', 'oven','toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
              'hair drier', 'toothbrush']


tracker=Tracker()

cap=cv2.VideoCapture('highway.mp4')

down = {}
up = {}
counter_down = []
counter_up = []
tickets = {}
count = 0
beep_timestamps = []

red_line_y = 198
blue_line_y = 268
offset = 6

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_no_audio.avi', fourcc, 20.0, (1020, 500))


# Generate a beep sound (0.1-second duration at 1000 Hz)
beep = Sine(1000).to_audio_segment(duration=100)
beep.export("beep.wav", format="wav")

fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second


while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    d = results[0].boxes.data
    d = d.detach().cpu().numpy()
    px = pd.DataFrame(d).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c or 'truck' in c or 'bus' in c:
            list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)

    yellow_color = (0, 255, 255)  # Yellow color for background
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (255, 0, 0)  # Blue color for lines

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           down[id]=time.time()   # current time when vehichle touch the first line
        if id in down:

           if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
             elapsed_time=time.time() - down[id]  # current time when vehicle touch the second line. Also we a re minusing the previous time ( current time of line 1)
             if counter_down.count(id)==0:
                counter_down.append(id)
                distance = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 36  # this will give kilometers per hour for each vehicle. This is the condition for going downside
                color = red_color if a_speed_kh > 40 else blue_color
                if a_speed_kh > 40:
                  tickets[id] = [time.time(), 'South', a_speed_kh]
                  beep_timestamps.append(count / fps)  # Record beep timestamp

                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), color, 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


        #####going UP blue line#####
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:

           if red_line_y<(cy+offset) and red_line_y > (cy-offset):
             elapsed1_time=time.time() - up[id]
             # formula of speed= distance/time
             if counter_up.count(id)==0:
                counter_up.append(id)
                distance1 = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms1 = distance1 / elapsed1_time
                a_speed_kh1 = a_speed_ms1 * 36
                if a_speed_kh1 > 40:
                  tickets[id] = [time.time(), 'North', a_speed_kh]
                  beep_timestamps.append(count / fps)  # Record beep timestamp

                color = red_color if a_speed_kh1 > 40 else blue_color
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), color, 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)




    text_color = (0, 0, 0)


    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)

    cv2.line(frame, (172, 198), (774, 198), yellow_color, 2)
    cv2.putText(frame, ('north Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), yellow_color, 2)
    cv2.putText(frame, ('south Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going South - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going North - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Save frame
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:
    #if cv2.waitKey(0) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()





0: 320x640 8 cars, 2 trucks, 330.2ms
Speed: 2.9ms preprocess, 330.2ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 321.8ms
Speed: 3.0ms preprocess, 321.8ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 342.1ms
Speed: 3.0ms preprocess, 342.1ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 326.4ms
Speed: 3.1ms preprocess, 326.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 310.4ms
Speed: 2.4ms preprocess, 310.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 496.6ms
Speed: 3.0ms preprocess, 496.6ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 504.7ms
Speed: 3.2ms preprocess, 504.7ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 504.9ms
Speed: 3.1

In [ ]:
# Load the video without sound
video = VideoFileClip("output_no_audio.avi")

# Load the beep sound
beep_clip = AudioFileClip("beep.wav")

# Create a list of beep sound clips at specific timestamps
beep_clips = []
for t in beep_timestamps:
    beep_clips.append(beep_clip.set_start(t).set_duration(beep_clip.duration))

# Combine all beep sounds into a single audio track
final_audio = CompositeAudioClip(beep_clips)

# Add the final audio to the video
final_video = video.set_audio(final_audio)

# Save the final video with beeps
final_video.write_videofile("output_with_beep.avi", codec="libx264", audio_codec="aac")

Moviepy - Building video output_with_beep.avi.
MoviePy - Writing audio in output_with_beepTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_with_beep.avi



Moviepy - Done !
Moviepy - video ready output_with_beep.avi


In [ ]:
for k,v in tickets.items():
  time, dir, speed = v
  print(f'ID: {k} in Time {time}, Moving {dir} caught driving {speed} Km\h')

ID: 13 in Time 1723207999.494555, Moving North caught driving 31.31153524063327 Km\h
ID: 14 in Time 1723208001.280498, Moving North caught driving 31.31153524063327 Km\h
ID: 5 in Time 1723208003.0807583, Moving South caught driving 47.676997544426115 Km\h
ID: 28 in Time 1723208019.203472, Moving North caught driving 47.676997544426115 Km\h
ID: 33 in Time 1723208029.264184, Moving South caught driving 41.365859859280135 Km\h
ID: 34 in Time 1723208033.219571, Moving North caught driving 41.365859859280135 Km\h
ID: 35 in Time 1723208039.5185559, Moving South caught driving 42.64551950463792 Km\h
ID: 40 in Time 1723208046.015935, Moving North caught driving 35.20428881866189 Km\h
ID: 51 in Time 1723208051.5156584, Moving North caught driving 28.932721065797566 Km\h
ID: 49 in Time 1723208053.8624828, Moving North caught driving 28.932721065797566 Km\h
ID: 55 in Time 1723208060.7649655, Moving North caught driving 28.932721065797566 Km\h
ID: 48 in Time 1723208063.2420073, Moving South caught